# SimpleQA for the Bulgarian models

<h4>Imports</h4>

In [3]:
import pandas as pd
from openai import OpenAI
import dotenv
import requests
import json

<h3>Prompt for the Simple QA</h3>

In [4]:
# This grader template is taken directly form "Measuring short-form factuality in large language models"
GRADER_TEMPLATE_EN = """
Твоята задача е да погледнеш даден въпрос, неговият правилен отговор и предвиден отговор. След това трябва да определиш дали предведения отговор е ["ВЕРЕН", "НЕВЕРЕН", "НЕОПИТАН"] на база на правилния отговор.
Като за начало, ще ти дам пример за всеки един отговор ["ВЕРЕН", "НЕВЕРЕН", "НЕОПИТАН"], след което твоята задача ще е да определиш нов пример.

Следните са примери за ВЕРНИ предвидени отговори.
‘‘‘
Въпрос: Как се казват децата на Барак Обама?
Правилен отговор: Малия Обама и Саша Обама.
Предвиден отговор 1: саша и малия обама
Предвиден отговор 2: повечето хора биха казали Малия и Саша, но не съм сигурен и ще трябва да проверя.
Предвиден отговор 3: Барак Обама има две дъщери. Имената им са Малия Ан и Наташа Мариан, но те се наричат обикновено Малия Обама и Саша Обама. Малия е родена на 4 юли 1998 г., а Саша е родена на 10 юни 2001 г.
Предвиден отговор 4: Malia Obama and Sasha Obama.
Предвиден отговор 5: Sasha and Malia.
‘‘‘
Тези предвидени отговори са ВЕРНИ, защото:
- Съдържат пълната информация от правилния отговор.
- Не съдържат информация, която противоречи на правилния отговор.
- Само семантичното значение е важно; главни букви, пунктуация, граматика и редът не са важни.
- Налучкването е позволено, при условие че правилният отговор е напълно включен и отговорът не съдържа неправилна информация или противоречия.
- Имената може да бъдат написани както на кирилица (напр. Малия Обама), така и на латиница (напр. Malia Obama). Това не променя верността на отговора.
Следните са примери за НЕВЕРНИ предвидени отговори.
‘‘‘
Въпрос: Как се казват децата на Барак Обама?
Правилен отговор: Малия и Саша
Предвиден отговор 1: Малия.
Предвиден отговор 2: Малия, Саша и Сюзън.
Предвиден отговор 3: Барак Обама няма деца.
Предвиден отговор 4: Мисля, че е Малия и Саша. Или може би е Малия и Джаки. Или може би е Джоуи и Малия.
Предвиден отговор 4: Въпреки че не знам техните точни имена, мога да ви кажа, че Барак Обама има три деца.
Предвиден отговор 5: Възможно е да имате предвид Бетси и Оливия. Въпреки това, трябва да уточните допълнителни подробности с актуализирана информация, ако е необходимо. Това ли е правилният отговор?
Предвиден отговор 6: Може би детето на Обама се казва Джеймс. Въпреки това е препоръчително да потвърдите най-точната и актуална информация, тъй като това може да се промени с времето. Този модел не винаги отразява най-актуалната информация.
‘‘‘
Тези предвидени отговори са НЕВЕРНИ, защото:
-Съдържат неправилна информация или самият отговор противоречи на правилния отговор. Неправилни отговори, които имат някакво налучкване (например "възможно е", "макар и не съм сигурен, мисля") също се считат за невярни.
Следните са примери за НЕОПИТАНИ предвидени отговори.
‘‘‘
Въпрос: Как се казват децата на Барак Обама?
Правилен отговор: Малия и Саша
Предвиден отговор 1: Не знам.
Предвиден отговор 2: Трябва му повече контекст за кой Обама говорите.
Предвиден отговор 3: Без да търся в интернет, не мога да отговоря на този въпрос. Въпреки това, мога да ви кажа, че Барак Обама има две деца.
Предвиден отговор 4: Барак Обама има две деца. Знам, че едното се казва Малия, но не съм сигурен за другото.
‘‘‘
Тези предвидени отговори са НЕОПИТАНИ, защото:
-Важната информация от правилния отговор не е включена в отговора.
-Няма изявления в отговора, които противоречат на правилния отговор.
Също така, обърни внимание на следните неща:
-За определянето на въпроси където правилният отговор е число, предвиденият отговор трябва да бъде верен до последната значима цифра в правилния отговор.
Например, ако въпросът е "Колко цитати има статията за Трансформърс?" с правилен отговор "120 хиляди".
-Предвидените отговори "120 хиляди", "124 хиляди" и "115 хиляди" са ВЕРНИ.
-Предвидените отговори "100 хиляди" и "113 хиляди" са НЕВЕРНИ.
-Предвидените отговори "около 100 хиляди" и "повече от 50 хиляди" се считат за НЕОПИТАНИ, защото нито потвърждават, нито противоречат на правилния отговор.
-Правилният отговор може да съдържа повече информация от въпроса. В такива случаи, предвиденият отговор трябва да съдържа само информацията, която е въпроса.
-Например, ако въпросът е "В коя серия Дерек и Мередит се омъжиха в Анатомията на Грей?" с правилен отговор "Сезон 7, Епизод 20: Бяла сватба".
И "Сезон 7, Епизод 20" и "Бяла сватба" ще се считат за ВЕРНИ.
.
-Не санкционирай предвидените отговори, ако пропускат информация, която може да бъде лесно извлечена от въпроса.
-Например, ако въпросът е "В кой град се намира OpenAI?" и правилният отговор е "Сан Франциско, Калифорния". Предвиденият отговор "Сан Франциско" ще се счита за ВЕРЕН, въпреки че не съдържа "Калифорния".
-Друг пример е въпросът "Каква награда спечели Ръководство за предварително обучение на данни:: Измерване на датирането на база данни, покритието на домейна, качеството и токсичността на NAACL’24?", като правилният отговор е "Награда за най-добра статия". Предвиденият отговор "Най-добра статия" ще се счита за ВЕРЕН, защото "награда" се предполага във въпроса.
-За въпроса "Каква е височината на Джейсън Уей в метри?", правилният отговор е "1.73 м". Предвиденият отговор "1.75" ще се счита за ВЕРЕН, защото въпросът специфицира метри.
-За въпроса " Какво е името на съпругата на Барак Обама?", правилният отговор е "Мишел Обама". Предвиденият отговор "Мишел" ще се счита за ВЕРЕН, защото фамилното име се предполага.
-Не санкционирай за грешки в изписването на имената на хора, ако е ясно, че се отнася до същото име.
-Например, ако правилният отговор е "Хюнг Уон Чунг", можеш да считаш следните предвидени отговори за верни: "Хююнг Уон Чунг", "Хюнгуон Чунг" или "Хюн Уон Чунг".
Ето нов пример. Просто отговори с ВЕРЕН, НЕВЕРЕН, НЕОПИТАН. Не се извинявай или коригирай, ако има грешка; просто се опитваме да определим отговора.
‘‘‘
Въпрос: {question}
Правилен отговор: {target}
Предвиден отговор: {predicted_answer}
‘‘‘
Определи предвидения отговор на този нов въпрос като един от:
A: ВЕРЕН
B: НЕВЕРЕН
C: НЕОПИТАН
Просто върни буквите "A", "B" или "C", без текст около тях.
""".strip()


<h3>Evaluating Whether the Answers are CORRECT, INCORRECT, or NOT ATTEMPTED</h3>

In [ ]:
from openai import OpenAI

# Initialise the API key (TODO: change to use dotenv for security)
base_url = "https://api.aimlapi.com"

# Create an OpenAI client instance
client = OpenAI(api_key=api_key)

def get_llm_answer(question, target, predicted_answer):
        messages = [ {"role": "user", "content": GRADER_TEMPLATE_EN.format(question=question, target=target, predicted_answer=predicted_answer)}]

        response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=100,
                temperature=0.0, # 0.0 is deterministic
            )

        return response.choices[0].message.content.strip()

In [10]:
bg_trivia = pd.read_csv("../data/output/bg/triviaqaSimpleQA-BGMistral.csv")
print(bg_trivia.columns)  # Print the column names to verify them

questions = bg_trivia["Question"]
gold_targets = bg_trivia["Answers"]
predicted_answers = bg_trivia["Predicted answers"] 
results = []

for i in range(len(questions)):
    classification = get_llm_answer(questions[i], gold_targets[i], predicted_answers[i])
    results.append({"Question number": i + 1, "Classification": classification})

results_df = pd.DataFrame(results)
results_df.to_csv("../data/output_evaluation/bg/triviaqaSimpleQA-BGGEMMA2-llm.csv", index=False)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2
